# NIRCam Imaging Saturation

Calculate saturation in each NIRCam filter in:
* full detector
* 64x64 subarray
* 160x160 subarray with weak lens WLP8

JDox: [NIRCam Bright Source Limits](https://jwst-docs.stsci.edu/jwst-near-infrared-camera/nircam-performance/nircam-bright-source-limits)
 
*** 
<div class="alert alert-warning">

Requires installation of the JWST ETC Pandeia Python engine:
    
https://jwst-docs.stsci.edu/jwst-exposure-time-calculator-overview/jwst-etc-pandeia-engine-tutorial 

</div>

***

## Imports

In [1]:
import os
import numpy as np  
#import json
import astropy.io.ascii

In [3]:
from pandeia.engine.perform_calculation import perform_calculation
from pandeia.engine.calc_utils import build_default_calc

In [13]:
filter_table = astropy.io.ascii.read('NIRCam_filters.txt', format='commented_header', header_start=-1)
filter_table

filter,center,bandwidth,response,lambda1,lambda2
str6,float64,float64,float64,float64,float64
F070W,0.704,0.128,0.237,0.624,0.781
F090W,0.901,0.194,0.318,0.795,1.005
F115W,1.154,0.225,0.333,1.013,1.282
F140M,1.404,0.142,0.434,1.331,1.479
F150W,1.501,0.318,0.476,1.331,1.668
F162M,1.626,0.168,0.469,1.542,1.713
F164N,1.644,0.02,0.385,1.635,1.653
F150W2,1.671,1.227,0.489,1.007,2.38
F182M,1.845,0.238,0.505,1.722,1.968


In [19]:
filters = list(filter_table['filter'])
filters = [filt.lower() for filt in filters]
filters

['f070w',
 'f090w',
 'f115w',
 'f140m',
 'f150w',
 'f162m',
 'f164n',
 'f150w2',
 'f182m',
 'f187n',
 'f200w',
 'f210m',
 'f212n',
 'f250m',
 'f277w',
 'f300m',
 'f322w2',
 'f323n',
 'f335m',
 'f356w',
 'f360m',
 'f405n',
 'f410m',
 'f430m',
 'f444w',
 'f460m',
 'f466n',
 'f470n',
 'f480m']

In [10]:
#WLP8_TS_filters  = '      F140M F150W F182M F187N F200W F210M F212N'.split()  # Time Series Imaging
#WLP8_GTS_filters = 'F070W F140M       F182M F187N       F210M F212N'.split()  # Grism Time Series Imaging
wl_filters = 'f070w f140m f150w f182m f187n f200w f210m f212n'.split()

In [25]:
def check_for_saturation(filt, mag, weak_lens=False):
    global calculation
    calculation['configuration']['instrument']['filter'] = filt.lower()
    calculation['scene'][0]['spectrum']['normalization']['norm_flux'] = mag
    
    lam = int(filt[1:4]) / 100.  # good enough here
    channel = ['sw', 'lw'][lam > 2.4]
    calculation['configuration']['instrument']['mode'] = channel + '_ts'  # time-series imaging
    if weak_lens:
        if filt == 'f070w':
            calculation['configuration']['instrument']['aperture'] = 'wlp8__tsgrism'
        else:
            calculation['configuration']['instrument']['aperture'] = 'wlp8__ts'
    else:
        calculation['configuration']['instrument']['aperture'] = channel

    results = perform_calculation(calculation)
    sat = results['1d']['n_full_saturated'][1][0]
    return sat

## Test calculation

https://jwst-docs.stsci.edu/jwst-exposure-time-calculator-overview/jwst-etc-pandeia-engine-tutorial/pandeia-quickstart#PandeiaQuickstart-Observingmodes

config_jwst.js file lists all available options

In [8]:
#calculation = build_default_calc("jwst", "nircam", "sw_imaging")
calculation = build_default_calc("jwst", "nircam", "sw_ts")

#calculation['configuration']['detector']['subarray'] = 'sub64p'
calculation['configuration']['detector']['subarray'] = 'sub160p'  # weak lens WLP8
#calculation['configuration']['detector']['subarray'] = 'full'

calculation['configuration']['instrument']['aperture'] = 'wlp8__ts'
#calculation['configuration']['instrument']['aperture'] = 'sw'

calculation['configuration']['detector']['readout_pattern'] = 'rapid'
calculation['configuration']['detector']['ngroup'] = 2
calculation['configuration']['detector']['nint'] = 1
calculation['configuration']['detector']['nexp'] = 1

calculation['configuration']['instrument']['filter'] = 'f200w'

calculation['scene'][0]['spectrum']['sed'] = {'key':'g2v', 'sed_type':'phoenix'}

calculation['scene'][0]['spectrum']['normalization'] = {'type':'photsys',
                                                       'norm_fluxunit':'vegamag',
                                                        'bandpass':'bessell,k'}
mag = 3  # F200W WLP8 SUB160P
#mag = 13  # F200W FULL
calculation['scene'][0]['spectrum']['normalization']['norm_flux'] = mag

calculation['strategy']['background_subtraction'] = False

In [9]:
results = perform_calculation(calculation)
#results['1d']
lam, sat = results['1d']['n_full_saturated']
sat[0]

/Users/dcoe/miniconda3/envs/pandeia_dev_11_07_2022/lib/python3.8/site-packages/pandeia.engine-1.8.dev0-py3.8.egg/pandeia/engine/report.py:637: RuntimeWarning: divide by zero encountered in double_scalars
  self.r['scalar']['sn'] = signal/noise
/Users/dcoe/miniconda3/envs/pandeia_dev_11_07_2022/lib/python3.8/site-packages/pandeia.engine-1.8.dev0-py3.8.egg/pandeia/engine/report.py:921: RuntimeWarning: divide by zero encountered in log
  if np.log(abs(val)) < -1*precision and val != 0.0:


16

# Calculate saturation in each filter

In [38]:
def calculate_saturations(filters, subarray, weak_lens=False, save_output=True, verbose=False):
    # Converge on saturation magnitude splitting the difference by 2 each time
    v2_sat_mags = []
    tolerance = 0.01
    calculation['configuration']['detector']['subarray'] = subarray
    for filt in filters:
        if verbose: print(filt)
        mag  = 5
        dmag = 10
        sat0 = check_for_saturation(filt, mag, weak_lens)
        sat = sat0
        if verbose: print(sat, mag)
        while dmag > tolerance / 4:
            dmag_sign = [-1, 1][sat > 0]
            mag += dmag_sign * dmag
            sat = check_for_saturation(filt, mag, weak_lens)
            if verbose: print(sat, mag)
            dmag /= 2
        v2_sat_mags.append(mag)
        if verbose:
            print('--------------')
        else:
            print(filt.ljust(7), '%5.2f' % mag)

    if save_output:
        outfile = 'nircam_saturation_%s' % subarray
        if weak_lens:
            outfile += '_wlp8'
        outfile += '.txt'
        print('SAVING', outfile)
        astropy.io.ascii.write([filters, v2_sat_mags], outfile, names='filt mag'.split())
        
    return v2_sat_mags

In [40]:
v2_sat_mags = calculate_saturations(filters, 'sub64p', False)

/Users/dcoe/miniconda3/envs/pandeia_dev_11_07_2022/lib/python3.8/site-packages/pandeia.engine-1.8.dev0-py3.8.egg/pandeia/engine/report.py:637: RuntimeWarning: divide by zero encountered in double_scalars
  self.r['scalar']['sn'] = signal/noise
/Users/dcoe/miniconda3/envs/pandeia_dev_11_07_2022/lib/python3.8/site-packages/pandeia.engine-1.8.dev0-py3.8.egg/pandeia/engine/report.py:921: RuntimeWarning: divide by zero encountered in log
  if np.log(abs(val)) < -1*precision and val != 0.0:
/var/folders/y3/vsjr6j554mnfpgy_m1xrnckm0003tz/T/ipykernel_87091/2787465782.py:14: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  dmag_sign = [-1, 1][sat > 0]


f070w   10.15
f090w   10.44
f115w   10.36
f140m    9.47
f150w   10.23
f162m    9.29
f164n    6.87
f150w2  11.59
f182m    9.13
f187n    6.48
f200w    9.55
f210m    8.39
f212n    6.04
f250m    8.63
f277w    9.77
f300m    8.61
f322w2  10.25
f323n    5.59
f335m    8.28
f356w    9.01
f360m    8.05
f405n    5.03
f410m    7.56
f430m    6.67
f444w    8.25
f460m    6.11
f466n    4.14
f470n    4.00
f480m    6.24
SAVING nircam_saturation_sub64p.txt


In [36]:
v2_sat_mags = calculate_saturations(filters, 'full', False)

/var/folders/y3/vsjr6j554mnfpgy_m1xrnckm0003tz/T/ipykernel_87091/2018796526.py:15: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  dmag_sign = [-1, 1][sat > 0]


f070w   15.98
f090w   16.27
f115w   16.19
f140m   15.29
f150w   16.06
f162m   15.12
f164n   12.69
f150w2  17.41
f182m   14.96
f187n   12.30
f200w   15.38
f210m   14.22
f212n   11.87
f250m   14.46
f277w   15.60
f300m   14.43
f322w2  16.08
f323n   11.42
f335m   14.11
f356w   14.84
f360m   13.88
f405n   10.85
f410m   13.39
f430m   12.50
f444w   14.07
f460m   11.94
f466n    9.96
f470n    9.83
f480m   12.06
SAVING nircam_saturation_full.txt


In [39]:
v2_sat_mags = calculate_saturations(wl_filters, 'sub160p', True)

/Users/dcoe/miniconda3/envs/pandeia_dev_11_07_2022/lib/python3.8/site-packages/pandeia.engine-1.8.dev0-py3.8.egg/pandeia/engine/report.py:921: RuntimeWarning: divide by zero encountered in log
  if np.log(abs(val)) < -1*precision and val != 0.0:
/var/folders/y3/vsjr6j554mnfpgy_m1xrnckm0003tz/T/ipykernel_87091/2787465782.py:14: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  dmag_sign = [-1, 1][sat > 0]
/Users/dcoe/miniconda3/envs/pandeia_dev_11_07_2022/lib/python3.8/site-packages/pandeia.engine-1.8.dev0-py3.8.egg/pandeia/engine/report.py:637: RuntimeWarning: divide by zero encountered in double_scalars
  self.r['scalar']['sn'] = signal/noise


f070w    1.89
f140m    2.54
f150w    3.43
f182m    2.82
f187n    0.40
f200w    3.50
f210m    2.62
f212n    0.33
SAVING nircam_saturation_sub160p_wlp8.txt
